## Weighted Jaccard Similarity
Weighted Jaccard similarity computes the Jaccard similarity, taking into account weights placed on the nodes of the graph.

To compute the weighted Jaccard similarity between each pair of vertices connected by an edge in cuGraph use:
**nvJaccard_w(input_graph, vect_weights_ptr)**
* input_graph: A cugraph.Graph object
* vect_weights_ptr: An array of vertex weights

Returns: df: cudf.DataFrame with three columns:
* df['source']: The source vertex id.
* df['destination']: The destination vertex id.
* df['jaccard_coeff']: The weighted jaccard coefficient computed between the source and destination vertex.

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

This is a small graph which allows for easy visual inspection to validate results.  

### Edge Weights
Since the test graph does not include weights, for this example we will compute and use the PageRank score as the weights

In [ ]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

## Read the data using cuDF

In [ ]:
# Test file  - using the clasic Karate club dataset.  
datafile='../data/networks/karate-data.csv'

In [ ]:
# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()), )

In [ ]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"])

### Compute PageRank to use as weights

In [ ]:
# Call Pagerank on the graph to get weights to use:
pr_df = cugraph.pagerank(G)

### Now compute the Weighted Jaccard 

In [ ]:
# Call weighted Jaccard using the Pagerank scores as weights:
df = cugraph.nvJaccard_w(G1, pr_df['pagerank'])

In [ ]:
# Find the most similar pair of vertices
bestEdge = 0
for i in range(len(df)):
    if df['jaccard_coeff'][i] > df['jaccard_coeff'][bestEdge]:
        bestEdge = i
print("Vertices " + str(df['source'][bestEdge]) + " and " + str(df['destination'][bestEdge]) + " are most similar with score: " + str(df['jaccard_coeff'][bestEdge]))

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___